# 0.  结论：大量 函数只进不出 原因  是 if无花括号包裹的return


1. 大量 函数进入 无 对应 函数返回 的主要原因 是 存在 约600个函数 中有 if无花括号包裹的return

2. **if无花括号包裹的return  是导致 函数只进不出问题 的 主要原因**

3. if无花括号包裹的return  举例:

```cpp
if(...)  //没有花括号的if
    return;//紧挨着就return

//这种语句，被我人为跳过了，因此要加花括号，才能在if和return之间插入语句。

```

#  1. 产生tick日志

#####  编译最小main : 文件 mini_main.c
```cpp
//文件 mini_main.c
int main(int argc, char** argv){
    return 0;
}
```

```bash
tick_save=true /pubx/build-llvm15/bin/clang-15 -c mini_main.c
#生成tick日志文件:
#-rw-rw-r--  131M  8月  1 22:49 clang-15_78152_1690901340709_1
```

# 2. 加载tick日志

In [1]:
# %config InlineBackend.figure_format = 'svg'
%config InlineBackend.rc={'figure.figsize': (30,20)}

In [2]:
import numpy 
import math

In [3]:
import seaborn

In [4]:
import plotly.offline as plotly_offline
plotly_offline.init_notebook_mode(connected=True)          
import plotly.graph_objs as graph_objs                
import plotly.figure_factory as figure_factory 

from plotly.graph_objs import Scatter, Figure, Scatter3d

import matplotlib.pyplot



In [5]:
import pandas
# pandas.options.display.max_columns = None
# pandas.options.display.max_rows = None




In [6]:
%%bash
ls -lhrt /tick_data_home/

总计 545M
-rw-rw-r-- 1 zz zz  20M  7月 27 16:42 clang-15_252255_1690447364166_1
-rw-rw-r-- 1 zz zz  17M  7月 27 17:54 clang-15_268265_1690451684154_1
-rw-rw-r-- 1 zz zz  23M  7月 27 18:35 clang-15_272726_1690454100567_1
-rw-rw-r-- 1 zz zz  19M  7月 27 19:08 clang-15_279683_1690456088122_1
-rw-rw-r-- 1 zz zz 5.0M  7月 27 19:08 clang-15_279682_1690456088057_1
-rw-rw-r-- 1 zz zz  22M  7月 27 19:08 clang-15_279758_1690456131168_1
-rw-rw-r-- 1 zz zz 5.0M  7月 27 19:08 clang-15_279757_1690456131110_1
-rw-rw-r-- 1 zz zz  66M  7月 27 19:40 clang-15_284358_1690458014273_1
-rw-rw-r-- 1 zz zz  20M  7月 29 23:18 clang-15_320593_1690643890944_1
-rw-rw-r-- 1 zz zz  18M  7月 29 23:45 clang-15_406251_1690645502530_1
-rw-rw-r-- 1 zz zz  32M  7月 30 22:15 clang-15_297928_1690726545091_1
-rw-rw-r-- 1 zz zz  32M  7月 30 22:17 clang-15_297999_1690726628322_1
-rw-rw-r-- 1 zz zz  32M  7月 30 22:17 clang-15_298040_1690726675727_1
-rw-rw-r-- 1 zz zz  27M  7月 30 23:02 clang-15_303915_1690729326061_1
-rw-rw-r-- 1 zz zz  37M  7

In [7]:
#编译正常: mini_main.c
df=pandas.read_csv(filepath_or_buffer="/tick_data_home/clang-15_78152_1690901340709_1",sep=',' , quotechar="'")


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968189 entries, 0 to 968188
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   滴答           968189 non-null  int64 
 1   tickKind     968189 non-null  int64 
 2   funcEnterId  968189 non-null  int64 
 3   rTSVarC      968189 non-null  int64 
 4   d栈生          968189 non-null  int64 
 5   d栈死          968189 non-null  int64 
 6   d堆生          968189 non-null  int64 
 7   d堆死          968189 non-null  int64 
 8   栈生           968189 non-null  int64 
 9   栈死           968189 non-null  int64 
 10  栈净           968189 non-null  int64 
 11  堆生           968189 non-null  int64 
 12  堆死           968189 non-null  int64 
 13  堆净           968189 non-null  int64 
 14  srcFile      968189 non-null  object
 15  funcLine     968189 non-null  int64 
 16  funcCol      968189 non-null  int64 
 17  funcName     957698 non-null  object
dtypes: int64(16), object(2)
memory usage: 133.0+

In [9]:
df.tail()

,滴答,tickKind,funcEnterId,rTSVarC,d栈生,d栈死,d堆生,d堆死,栈生,栈死,栈净,堆生,堆死,堆净,srcFile,funcLine,funcCol,funcName
968184,844305,0,121943,1,1,0,0,0,239868,218343,21525,0,0,0,/pubx/llvm-project/llvm/lib/Support/PrettyStac...,195,39,printForSigInfoIfNeeded
968185,844306,0,121943,1,0,0,0,0,239868,218343,21525,0,0,0,/pubx/llvm-project/llvm/lib/Support/PrettyStac...,195,39,printForSigInfoIfNeeded
968186,844307,2,121943,1,0,1,0,0,239868,218343,21525,0,0,0,/pubx/llvm-project/llvm/lib/Support/PrettyStac...,195,39,printForSigInfoIfNeeded
968187,844308,2,121942,0,0,0,0,0,239868,218344,21524,0,0,0,/pubx/llvm-project/llvm/lib/Support/PrettyStac...,227,49,llvm::PrettyStackTraceEntry::~PrettyStackTrace...
968188,843973,1,121780,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/MemAlloc.cpp,24,72,llvm::deallocate_buffer


# 3.  过滤出只有 函数进入 没有 函数返回 的tick日志

```cpp
// 文件: http://192.168.1.4:10880/analyze_code/clang-ctk/src/75700ddaacb8f7c1e19be8cabcafefe2831bd5b5/t_clock_tick/t_clock_tick.cpp

enum TickKind{
    //正常tick
    NormalTick=0,
    //函数进入tick 只作为 和 函数返回tick 做比对，看哪里少插入了X__funcReturn
    FuncEnter=1,
    //函数返回tick
    FuncReturn=2

};
```

In [10]:
#函数进入tick
FuncEnter=1
#函数返回tick
FuncReturn=2

##  3.1  函数进入tick 或 函数返回tick 的日志行

In [11]:
#函数进入tick 或 函数返回tick 的日志行
df_ER = df[df['tickKind'].isin( [FuncEnter, FuncReturn] )]
df_ER.shape

(225693, 18)

In [12]:
df_ER.info()

<class 'pandas.core.frame.DataFrame'>
Index: 225693 entries, 0 to 968188
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   滴答           225693 non-null  int64 
 1   tickKind     225693 non-null  int64 
 2   funcEnterId  225693 non-null  int64 
 3   rTSVarC      225693 non-null  int64 
 4   d栈生          225693 non-null  int64 
 5   d栈死          225693 non-null  int64 
 6   d堆生          225693 non-null  int64 
 7   d堆死          225693 non-null  int64 
 8   栈生           225693 non-null  int64 
 9   栈死           225693 non-null  int64 
 10  栈净           225693 non-null  int64 
 11  堆生           225693 non-null  int64 
 12  堆死           225693 non-null  int64 
 13  堆净           225693 non-null  int64 
 14  srcFile      225693 non-null  object
 15  funcLine     225693 non-null  int64 
 16  funcCol      225693 non-null  int64 
 17  funcName     216079 non-null  object
dtypes: int64(16), object(2)
memory usage: 32.7+ MB


## 3.2 函数进入tick行数, 函数返回tick行数

In [13]:
# 按照tickKind字段分组，并获取每个组的行数
grouped_df_ER = df_ER.groupby('tickKind').size().reset_index()

# 将列名设置为"tickKind"和"行数"
grouped_df_ER.columns = ['tickKind', '该tickKind组内tick行数']

# 创建一个新的DataFrame，将"tickKind"和"行数"作为列添加到新的DataFrame中
g_df_ER = pandas.DataFrame(grouped_df_ER)

g_df_ER

,tickKind,该tickKind组内tick行数
0,1,121945
1,2,103748


## 3.3  存在 大量 函数进入 缺少 对应 函数返回

### 3.3.1 其内只有一行日志的函数进入id们

In [14]:
# 按照funcEnterId字段分组，并保留组内行数等于1行的组
_df_ER_1 = df_ER.groupby('funcEnterId').filter(lambda x: len(x) == 1)

# 重置索引，形成新的DataFrame
df_ER_1 = _df_ER_1.reset_index(drop=True)

df_ER_1.shape

(18345, 18)

In [15]:
df_ER_1.head(5)

,滴答,tickKind,funcEnterId,rTSVarC,d栈生,d栈死,d堆生,d堆死,栈生,栈死,栈净,堆生,堆死,堆净,srcFile,funcLine,funcCol,funcName
0,52973,1,7113,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp
1,52991,1,7115,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp
2,53014,1,7118,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp
3,53037,1,7121,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp
4,53060,1,7124,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp


### 3.3.2 其内只有一行日志的函数进入id们  都是 函数进入

In [16]:
_df_ER_1_enter = df_ER.groupby('funcEnterId').filter(lambda x: len(x) == 1 and x.tickKind == FuncEnter)

df_ER_1_enter = _df_ER_1_enter.reset_index(drop=True)

df_ER_1_enter.shape

(18345, 18)

In [17]:
df_ER_1_enter.head(5)

,滴答,tickKind,funcEnterId,rTSVarC,d栈生,d栈死,d堆生,d堆死,栈生,栈死,栈净,堆生,堆死,堆净,srcFile,funcLine,funcCol,funcName
0,52973,1,7113,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp
1,52991,1,7115,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp
2,53014,1,7118,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp
3,53037,1,7121,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp
4,53060,1,7124,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp


### 3.3.3 其内只有一行日志的函数进入id们  都不是 函数返回

In [18]:
_df_ER_k1_return = df_ER.groupby('funcEnterId').filter(lambda x: len(x) == 1 and x.tickKind == FuncReturn)

df_ER_k1_return = _df_ER_k1_return.reset_index(drop=True)

df_ER_k1_return.shape

(0, 18)

In [19]:
df_ER_k1_return.head(5)

,滴答,tickKind,funcEnterId,rTSVarC,d栈生,d栈死,d堆生,d堆死,栈生,栈死,栈净,堆生,堆死,堆净,srcFile,funcLine,funcCol,funcName


### 3.3.4  形象化: 函数进入 缺少 对应 函数返回

In [20]:
df_ER_1.head(5)

,滴答,tickKind,funcEnterId,rTSVarC,d栈生,d栈死,d堆生,d堆死,栈生,栈死,栈净,堆生,堆死,堆净,srcFile,funcLine,funcCol,funcName
0,52973,1,7113,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp
1,52991,1,7115,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp
2,53014,1,7118,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp
3,53037,1,7121,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp
4,53060,1,7124,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp


In [21]:
例子函数进入id=df_ER_1.funcEnterId.values[0]
例子函数进入id

7113

In [22]:
df[ df['funcEnterId'].isin( [例子函数进入id] ) &  df['tickKind'].isin( [FuncEnter, FuncReturn] ) ]


,滴答,tickKind,funcEnterId,rTSVarC,d栈生,d栈死,d堆生,d堆死,栈生,栈死,栈净,堆生,堆死,堆净,srcFile,funcLine,funcCol,funcName
60206,52973,1,7113,0,0,0,0,0,0,0,0,0,0,0,/pubx/llvm-project/llvm/lib/Support/regcomp.c,437,1,p_ere_exp



> 
>  人工检查/pubx/llvm-project/llvm/lib/Support/regcomp.c	 中的 此行列位置，发现 主要是 不带花括号的if return造成的。

```cpp
if(...)  //没有花括号的if
    return;//紧挨着就return

//这种语句，被我人为跳过了，因此要加花括号，才能在if和return之间插入语句。

```

## 3.4  哪些函数 只有   函数进入  却没有  函数返回 

In [23]:
# 按照funcEnterId字段分组，并保留组内行数等于1行的组
df_ER_k1__ = df_ER.groupby('funcEnterId').filter(lambda x: len(x) == 1)

# 重置索引，形成新的DataFrame df_ER_k1
df_ER_k1_ = df_ER_k1__.reset_index(drop=True)

# 对df_ER_k1按funcName字段分组，并输出各不同的funcName
只进不出函数名 = df_ER_k1_.groupby('funcName').size().reset_index()
只进不出函数名.columns = ['funcName', '进入次数']

只进不出函数名_进入次数倒序=只进不出函数名.sort_values(by='进入次数', ascending=False)

# pandas.set_option('display.max_rows', None)
# pandas.set_option('display.max_colwidth', None)


只进不出函数名_进入次数倒序 = 只进不出函数名_进入次数倒序.reset_index(drop=True)
只进不出函数名_进入次数倒序.shape


(590, 2)

In [24]:
只进不出函数名_进入次数倒序.head(5)

,funcName,进入次数
0,clang::LangOptions::getOpenCLCompatibleVersion,3263
1,clang::Lexer::getSourceLocation,1973
2,llvm::opt::OptTable::getOption,1358
3,clang::Lexer::LexTokenInternal,1171
4,clang::LineTableInfo::FindNearestLineEntry,752


>  上面 按 funcName 分组， 不易排查问题，下面 加入 字段  源文件名、函数所在行号、函数所在列号 参与分组， 

> funcName 已经是 函数名 主键了 ，新加字段 都不是 函数名 主键，  理论上不影响结果

In [25]:
# 按照funcEnterId字段分组，并保留组内行数等于1行的组
df_ER_k1__ = df_ER.groupby('funcEnterId').filter(lambda x: len(x) == 1)

# 重置索引，形成新的DataFrame df_ER_k1
df_ER_k1_ = df_ER_k1__.reset_index(drop=True)

# 对df_ER_k1按funcName字段分组，并输出各不同的funcName
只进不出函数名_带函数位置 = df_ER_k1_.groupby( ['funcName','srcFile', 'funcLine', 'funcCol']).size().reset_index()
只进不出函数名_带函数位置.columns = ['funcName', 'srcFile', 'funcLine', 'funcCol', '进入次数']

只进不出函数名_带函数位置_进入次数倒序=只进不出函数名_带函数位置.sort_values(by='进入次数', ascending=False)

pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_colwidth', None)


只进不出函数名_带函数位置_进入次数倒序 = 只进不出函数名_带函数位置_进入次数倒序.reset_index(drop=True)
只进不出函数名_带函数位置_进入次数倒序.shape


(594, 5)

In [26]:
只进不出函数名_带函数位置_进入次数倒序

,funcName,srcFile,funcLine,funcCol,进入次数
0,clang::LangOptions::getOpenCLCompatibleVersion,/pubx/llvm-project/clang/lib/Basic/LangOptions.cpp,55,58,3263
1,clang::Lexer::getSourceLocation,/pubx/llvm-project/clang/lib/Lex/Lexer.cpp,1149,64,1973
2,llvm::opt::OptTable::getOption,/pubx/llvm-project/llvm/lib/Option/OptTable.cpp,161,58,1358
3,clang::Lexer::LexTokenInternal,/pubx/llvm-project/clang/lib/Lex/Lexer.cpp,3518,76,1171
4,clang::LineTableInfo::FindNearestLineEntry,/pubx/llvm-project/clang/lib/Basic/SourceManager.cpp,241,71,752
5,clang::IdentifierInfo::getPPKeywordID,/pubx/llvm-project/clang/lib/Basic/IdentifierTable.cpp,324,59,748
6,getBuiltinDiagClass,/pubx/llvm-project/clang/lib/Basic/DiagnosticIDs.cpp,349,54,408
7,clang::MacroDirective::getDefinition,/pubx/llvm-project/clang/lib/Lex/MacroInfo.cpp,198,57,404
8,clang::DiagnosticsEngine::DiagStateMap::lookup,/pubx/llvm-project/clang/lib/Basic/Diagnostic.cpp,212,67,404
9,clang::DiagnosticIDs::isBuiltinExtensionDiag,/pubx/llvm-project/clang/lib/Basic/DiagnosticIDs.cpp,445,65,404


> **看一下无函数返回 的 函数进入 次数最多的 第一行  ， 确实 也是 if无花括号包裹的return 这种样子**

> **足以说明，if无花括号包裹的return  是导致 函数只进不出问题 的 主要原因**

```cpp
//文件:  /pubx/llvm-project/clang/lib/Basic/LangOptions.cpp
/*第55行*/unsigned LangOptions::getOpenCLCompatibleVersion() const {
  if (!OpenCLCPlusPlus)
    return OpenCLVersion;
  if (OpenCLCPlusPlusVersion == 100)
    return 200;
  if (OpenCLCPlusPlusVersion == 202100)
    return 300;
  llvm_unreachable("Unknown OpenCL version");
}
```



In [27]:

pandas.reset_option('display.max_rows')
pandas.reset_option('display.max_colwidth')